# Projekt 2

### Dobór parametrów

#### Liczba symulacji R

Dla estymatora  
$$
\hat{I} = \frac{1}{R}\sum_{i=1}^R Y_i
$$  
błąd standardowy wynosi  
$$
\mathrm{SE}(\hat{I}) = \frac{\sigma}{\sqrt{R}},
$$  
czyli dokładność rośnie powoli (proporcjonalnie do $1/\sqrt{R}$). Dobór $R$ musi więc równoważyć precyzję i koszt obliczeń.

W profesjonalnej wycenie opcji Monte Carlo używa się zwykle $10^4\!-\!10^5$ symulacji dla instrumentów o umiarkowanej wariancji wypłaty. Taki rząd wielkości zapewnia stabilność wyników i błędy standardowe na poziomie akceptowalnym w praktyce rynkowej.

W projekcie przyjęliśmy  
$$
R = 50{,}000,
$$  
co mieści się w typowym zakresie stosowanym w finansach i gwarantuje wiarygodne oszacowania przy rozsądnym czasie obliczeń. W metodach redukcji wariancji taka wartość jest nawet nadmiarowa, ponieważ ich estymatory mają mniejszą wariancję.

Warto zauważyć, że wartość $R = 50{,}000$, dobrana pod kątem opcji europejskiej (czyli bez bariery), jest tym bardziej wystarczająca w przypadku opcji z barierą. Jeśli oznaczymy wypłatę opcji europejskiej jako $A_\infty$ oraz wypłatę opcji z barierą $C$ jako $A_C$, to zachodzi  
$$
A_C = A_\infty \cdot \mathbf{1}_{\{\text{bariera nie została trafiona}\}},
$$  
czyli $A_C \le A_\infty$ dla każdej realizacji. Ponadto  
$$
A_C^2 \le A_\infty^2 \quad \text{oraz} \quad \mathbb{E}[A_C^2] < \mathbb{E}[A_\infty^2],
$$  
więc wariancja opcji barierowej spełnia  
$$
\mathrm{Var}(A_C) = \mathbb{E}[A_C^2] - \mathbb{E}[A_C]^2 < \mathbb{E}[A_\infty^2] - \mathbb{E}[A_\infty]^2 = \mathrm{Var}(A_\infty).
$$  
Zatem wariancja wypłat (czyli $\sigma^2$ używana w oszacowaniu błędu standardowego) jest mniejsza w przypadku opcji z barierą, co oznacza, że błąd standardowy $\mathrm{SE}(\hat{I})$ również jest mniejszy niż dla opcji europejskiej — przy tej samej liczbie symulacji $R$. Przyjęta wartość $R = 50{,}000$ jest więc tym bardziej wystarczająca dla przypadku z barierą.


#### dyskretyzacja (n)

Symulacje GBM obserwujemy w punktach
$$
t_k = \frac{k}{n},
$$
więc $n$ jest parametrem dyskretyzacji wpływającym na wykrywanie przekroczeń bariery.
Większe $n$ oznacza dokładniejszą obserwację ścieżki, ale też wyższy koszt obliczeń
(złożoność $\sim R \cdot n$).

W praktycznej wycenie opcji barierowych **nie stosuje się bariery ciągłej** — kontrakt
określa konkretne daty monitorowania, zwykle dzienne (ok. 250/rok), tygodniowe (ok. 52/rok)
lub miesięczne (ok. 12/rok). Profesjonalne modele odzwierciedlają właśnie taką dyskretną
strukturę, a nie idealizację ciągłą.

Traktując $T = 1$ jako rok, przyjęcie
$$
n = 50
$$
jest zgodne z **typową częstotliwością tygodniową** i odpowiada praktycznym standardom
wyceny dyskretnych barier, zapewniając jednocześnie rozsądny koszt obliczeń.

#### wartość bariery C

Wybór dwóch wartości bariery up-and-out — $H = 105$ oraz $H = 120$ — pozwala analizować dwa odmienne reżimy zachowania opcji barierowej przy założeniu $S_0 = 100$. Poziom $H = 105$ reprezentuje barierę bardzo bliską cenie początkowej, co skutkuje wysokim prawdopodobieństwem knock-outu i silną zależnością wyniku od maksymalnych wartości ścieżki. Taka konfiguracja jest szczególnie interesująca numerycznie, ponieważ ujawnia problemy związane z dyskretnym monitoringiem bariery oraz konieczność stosowania korekt, takich jak Brownian Bridge. Jednocześnie stanowi klasyczny przykład „silnie barierowy”, często wykorzystywany w literaturze do testowania jakości metod Monte Carlo.

Z kolei bariera $H = 120$ znajduje się znacznie dalej, co prowadzi do znacznie niższego prawdopodobieństwa aktywacji bariery i sprawia, że opcja zachowuje się bardziej jak instrument vanilla. Dzięki temu przypadek ten jest stabilniejszy numerycznie i pozwala badać, jak zmienia się wycena oraz charakterystyki opcji, gdy bariera ma marginalny wpływ na dynamikę wypłaty. Wspólne rozważenie obu poziomów umożliwia uchwycenie przejścia od silnego do słabego efektu bariery, dając pełniejszy obraz zjawiska i zgodnie z praktyką literaturową tworzy reprezentatywny zestaw przykładów do analiz teoretycznych i numerycznych.

# liczba strat $m$

W metodzie stratyfikacji ważne jest, aby w każdej stracie znajdowała się wystarczająco duża liczba próbek. Jeżeli całkowita liczba symulacji to $R$, to liczba próbek przypadających na jedną stratę wynosi

$$
R_i = \frac{R}{m}.
$$

Gdy $m$ jest zbyt duże, wtedy $R_i$ staje się małe, a średnie warunkowe $\bar{Y}_i$ obliczane w poszczególnych stratach mają wysoką wariancję. W skrajnym przypadku, kiedy $m$ byłoby zbliżone do $R$, każda strata miałaby około jednej próbki, przez co stratyfikacja przestaje działać i sprowadza się do zwykłego Monte Carlo.

Z kolei wyniki praktyczne i literatura dotycząca stratyfikacji dla wektorów normalnych pokazują, że większość redukcji wariancji uzyskuje się już dla niewielkich wartości $m$, typowo $m = 4$ lub $m = 8$. Zwiększanie liczby strat powyżej tego zakresu daje coraz mniejsze korzyści, a jednocześnie komplikuje implementację i analizę wyników.

Dlatego przy skalach obliczeń używanych w tym projekcie (np. $R$ rzędu dziesiątek tysięcy) naturalnym i efektywnym wyborem jest

$$
m = 4,
$$

co pozwala uzyskać dużą liczbę próbek w każdej stracie oraz wyraźną redukcję wariancji względem metody podstawowej.